In [78]:
import pandas as pd
import numpy as np
import random
import networkx as nx
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
import itertools
import csv
from sklearn.model_selection import train_test_split
from time import time
from node2vec import Node2Vec
from sklearn.preprocessing import StandardScaler
import lightgbm as lgbm
import optuna
from sklearn.metrics import accuracy_score

# Counting all nodes

In [79]:
nodes = dict()

with open("train-mod.txt") as file :
    end = file.seek(0, 2)
    file.seek(0)
    while file.tell() != end:
        line = file.readline().split()
        edges = list(itertools.combinations(line,2))
        for i in edges:
            if nodes.get(i) == None:
                node1 = i[0]
                node2 = i[1]
                if nodes.get((node2,node1)) == None:
                    nodes[i] = 1
                else:
                    nodes[(node2,node1)] += 1
            else:
                nodes[i] +=1

print(len(nodes))

16087


In [80]:
with open("weighted_graph.csv", "w", newline="") as a_file:

    writer = csv.writer(a_file)
    for key, value in nodes.items():
        writer.writerow([key[0], key[1], value])

    a_file.close()

In [81]:
g = nx.read_weighted_edgelist('weighted_graph.csv', delimiter=',', nodetype=int)

In [82]:
print(nx.info(g))

Name: 
Type: Graph
Number of nodes: 3816
Number of edges: 16087
Average degree:   8.4313
None


# Positive edges

In [84]:
edges_pos = list(nodes.keys())

In [85]:
with open("edges_pos_all.csv","w",newline="") as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(["Source","Target", "Label"])
    for edge in edges_pos:
        writer.writerow([edge[0], edge[1], 1])

# Generating negative edges (random sampling)

In [86]:
i = 0
num_test_edges = 16087
edges_neg = []
while i < num_test_edges:
    edge = random.sample(g.nodes(), 2)
    try:
        edge_exists = g.has_edge(edge[0],edge[1])
        if edge_exists == False:
            edges_neg.append([edge[0],edge[1]])
            i = i+1
    except Exception as e:
        pass

In [87]:
with open("edges_neg_16k.csv","w",newline="") as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(["Source","Target", "Label"])
    for edge in edges_neg:
        writer.writerow([edge[0], edge[1], 0])

# Train/Test split

In [89]:
edges_positive = pd.read_csv('edges_pos_all.csv').to_numpy()
edges_negative = pd.read_csv('edges_neg_16k.csv').to_numpy()

In [92]:
#reading total data df
df_pos = pd.DataFrame(edges_positive, columns=['source_node', 'destination_node', 'label'])
df_neg = pd.DataFrame(edges_negative, columns=['source_node', 'destination_node', 'label'])

In [95]:
df_pos_n2v =  df_pos[['source_node', 'destination_node']].copy()

In [97]:
# Generate walks
node2vec = Node2Vec(g, dimensions=30, walk_length=16, num_walks=50)

# train node2vec model
n2w_model = node2vec.fit(window=10, min_count=1)

Generating walks (CPU: 1): 100%|██████████| 50/50 [01:06<00:00,  1.34s/it]


In [98]:
data = pd.concat([df_pos, df_neg], ignore_index=True)

In [99]:
x_train_n2v = [(n2w_model[str(i)]+n2w_model[str(j)]) for i,j in zip(data['source_node'], data['destination_node'])]

<ipython-input-99-6d09be66ec01>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  x_train_n2v = [(n2w_model[str(i)]+n2w_model[str(j)]) for i,j in zip(data['source_node'], data['destination_node'])]


In [101]:
def generate_features(sample_list, test = False):
    features = []
    i = 0
    for sample in sample_list:
        #print(sample)
        source = sample[0]
        target = sample[1]
        if test == False:
            label = sample[2]
        else:
            label = -1
        
        feature = []
        try:
            i = i+1
            #print(i)
            
            #p = nx.common_neighbors(g, source, target)
            #feature.append(len(p))
            
            #p = nx.simrank_similarity(g, source, target)
            #feature.append(p)
            
            #preds = nx.resource_allocation_index(g, [(source, target)])
            #for u, v, p in preds:
            #    feature.append(p)

            preds = nx.jaccard_coefficient(g, [(source, target)])
            for u, v, p in preds:
                feature.append(p)

            preds = nx.adamic_adar_index(g, [(source, target)])
            for u, v, p in preds:
                feature.append(p)

            #preds = nx.preferential_attachment(g, [(source, target)])
            #for u, v, p in preds:
            #    feature.append(p)
            
            feature.append(label)  # append label
            
        except Exception as e:
            #print(e)
            pass
        features.append(feature)
    print("features: "+str(len(features)))
    return features

In [102]:
features_pos = generate_features(edges_positive)
features_neg = generate_features(edges_negative)

features: 16087
features: 16087


In [103]:
features = features_pos + features_neg

In [105]:
def write_train_to_csv(features):
    with open("train_16k_sim.csv","w",newline="") as csvfile:
        writer=csv.writer(csvfile)
        writer.writerow(["JC","AA","Label"])
        writer.writerows(features)
        
write_train_to_csv(features)

In [106]:
dataset_sim = pd.read_csv('train_16k_sim.csv')
FEATURE_SIZE=2

X_sim = dataset_sim.iloc[:,:FEATURE_SIZE].values
y_sim = dataset_sim.iloc[:,FEATURE_SIZE].values

In [109]:
x_train_n2v_1 = np.array(x_train_n2v)

In [112]:
# Feature Scaling
sc = StandardScaler()
X_sim_1 = sc.fit_transform(X_sim)

In [115]:
all_feats = np.concatenate((X_sim_1,x_train_n2v_1),axis=1)

In [118]:
%%time
def objective(trial):
    train_x, test_x, train_y, test_y = train_test_split(all_feats, data['label'], 
                                                test_size = 0.2, 
                                                random_state = 35)
    dtrain = lgbm.Dataset(train_x, label=train_y)
 
    param = {
        'objective': 'binary',
        'metric': 'auc',
        'is_unbalance': 'false',
        'boosting': trial.suggest_categorical('boosting', ['gbdt', 'dart']),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_uniform('subsample', 0.1, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 30),
        'max_bin': trial.suggest_int('max_bin', 25, 255),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.5)
    }
 
    gbm = lgbm.train(param, dtrain)
    preds = gbm.predict(test_x)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(test_y, pred_labels)
    return accuracy
 
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)
 
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-04-07 13:20:52,102] A new study created in memory with name: no-name-de56c568-7088-4cb6-8b5a-b98eb038d07f


[LightGBM] [Warning] bagging_fraction is set=0.48339248468054935, subsample=0.7378299496257708 will be ignored. Current value: bagging_fraction=0.48339248468054935
[LightGBM] [Warning] bagging_fraction is set=0.48339248468054935, subsample=0.7378299496257708 will be ignored. Current value: bagging_fraction=0.48339248468054935
[LightGBM] [Info] Number of positive: 12811, number of negative: 12928
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002070 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2944
[LightGBM] [Info] Number of data points in the train set: 25739, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497727 -> initscore=-0.009091
[LightGBM] [Info] Start training from score -0.009091
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2021-04-07 13:20:52,740] Trial 0 finished with value: 0.9586635586635587 and parameters: {'boosting': 'gbdt', 'lambda_l1': 0.024891537718029728, 'lambda_l2': 0.9691805918762091, 'num_leaves': 80, 'feature_fraction': 0.5553012538030526, 'bagging_fraction': 0.48339248468054935, 'bagging_freq': 6, 'min_child_samples': 72, 'subsample': 0.7378299496257708, 'max_depth': 12, 'max_bin': 92, 'learning_rate': 0.1961474698339416}. Best is trial 0 with value: 0.9586635586635587.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] bagging_fraction is set=0.874065718289452, subsample=0.645742419112278 will be ignored. Current value: bagging_fraction=0.874065718289452
[LightGBM] [Warning] bagging_fraction is set=0.874065718289452, subsample=0.645742419112278 will be ignored. Current value: bagging_fraction=0.874065718289452
[LightGBM] [Info] Number of positive: 12811, number of negative: 12928
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002727 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 864
[LightGBM] [Info] Number of data points in the train set: 25739, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497727 -> initscore=-0.009091
[LightGBM] [Inf

[I 2021-04-07 13:20:53,870] Trial 1 finished with value: 0.9613053613053613 and parameters: {'boosting': 'dart', 'lambda_l1': 0.006202224058052177, 'lambda_l2': 0.5241600786986856, 'num_leaves': 112, 'feature_fraction': 0.7197985944354747, 'bagging_fraction': 0.874065718289452, 'bagging_freq': 7, 'min_child_samples': 63, 'subsample': 0.645742419112278, 'max_depth': 24, 'max_bin': 27, 'learning_rate': 0.41228838644000915}. Best is trial 1 with value: 0.9613053613053613.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2021-04-07 13:27:37,681] Trial 298 finished with value: 0.9617715617715618 and parameters: {'boosting': 'gbdt', 'lambda_l1': 5.678359327738164e-08, 'lambda_l2': 8.435873850829699e-06, 'num_leaves': 195, 'feature_fraction': 0.7929924622935952, 'bagging_fraction': 0.8554620660661257, 'bagging_freq': 1, 'min_child_samples': 25, 'subsample': 0.5551681080159847, 'max_depth': 23, 'max_bin': 207, 'learning_rate': 0.32662862492748657}. Best is trial 201 with value: 0.9641025641025641.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2021-04-07 13:27:38,742] Trial 299 finished with value: 0.9634809634809635 and parameters: {'boosting': 'gbdt', 'lambda_l1': 1.3984024999884168e-07, 'lambda_l2': 6.409596383030565e-06, 'num_leaves': 196, 'feature_fraction': 0.8111245766449013, 'bagging_fraction': 0.8409284395200866, 'bagging_freq': 1, 'min_child_samples': 27, 'subsample': 0.6070138148801043, 'max_depth': 24, 'max_bin': 203, 'learning_rate': 0.3381122630090954}. Best is trial 201 with value: 0.9641025641025641.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [119]:
x_train, x_test, y_train, y_test = train_test_split(all_feats, data['label'], 
                                                test_size = 0.2, 
                                                random_state = 35)

In [120]:
train_data = lgbm.Dataset(x_train, y_train)
test_data = lgbm.Dataset(x_test, y_test)

# define parameters
parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'false',
    'boosting': 'dart', 
    'lambda_l1': 9.337045795886141e-07, 
    'lambda_l2': 5.882192529220549e-05, 
    'num_leaves': 216, 
    'feature_fraction': 0.7340991952157904, 
    'bagging_fraction': 0.9171352407891052, 
    'bagging_freq': 1, 
    'min_child_samples': 36, 
    'subsample': 0.5035121855349084, 
    'max_depth': 25, 
    'max_bin': 217, 
    'learning_rate': 0.35568718194758253
}

# train lightGBM model
model = lgbm.train(parameters,
                   train_data,
                   valid_sets=test_data,
                   num_boost_round=1000,
                   early_stopping_rounds=20)

[LightGBM] [Warning] bagging_fraction is set=0.9171352407891052, subsample=0.5035121855349084 will be ignored. Current value: bagging_fraction=0.9171352407891052
[LightGBM] [Warning] bagging_fraction is set=0.9171352407891052, subsample=0.5035121855349084 will be ignored. Current value: bagging_fraction=0.9171352407891052
[LightGBM] [Info] Number of positive: 12811, number of negative: 12928
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003201 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6944
[LightGBM] [Info] Number of data points in the train set: 25739, number of used features: 32
[LightGBM] [Warning] bagging_fraction is set=0.9171352407891052, subsample=0.5035121855349084 will be ignored. Current value: bagging_fraction=0.9171352407891052
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497727 -> initscore=-0.009091
[LightGBM] [Info] Start training from score -0.009091
[LightGBM] [Warni

/usr/local/Caskroom/miniconda/base/envs/smlproj/lib/python3.8/site-packages/lightgbm/callback.py:186: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')


[10]	valid_0's auc: 0.986563
[11]	valid_0's auc: 0.986769
[12]	valid_0's auc: 0.986795
[13]	valid_0's auc: 0.986752
[14]	valid_0's auc: 0.986808
[15]	valid_0's auc: 0.986907
[16]	valid_0's auc: 0.987503
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[17]	valid_0's auc: 0.987571
[18]	valid_0's auc: 0.987793
[19]	valid_0's auc: 0.987851
[20]	valid_0's auc: 0.987695
[21]	valid_0's auc: 0.987813
[22]	valid_0's auc: 0.987809
[23]	valid_0's auc: 0.988032
[24]	valid_0's auc: 0.987993
[25]	valid_0's auc: 0.987986
[26]	valid_0's auc: 0.988055
[27]	valid_0's auc: 0.988127
[28]	valid_0's auc: 0.988255
[29]	valid_0's auc: 0.988313
[30]	valid_0's auc: 0.988592
[31]	valid_0's auc: 0.98849
[32]	valid_0's auc: 0.988403
[33]	valid_0's auc: 0.988614
[34]	valid_0's auc: 0.9886
[35]	valid_0's auc: 0.988638
[36]	valid_0's auc: 0.988658
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[37]	valid_0's auc: 0.988572
[38]	valid_0's auc: 0.988548
[39]	valid_0

In [121]:
print('Saving model...')
# save model to file
model.save_model('JC_AA_n2v_LGBM_optuna_V1_300.txt')

Saving model...


In [122]:
testCols=['Id', 'source_node', 'destination_node'] 
df_test_public = pd.read_csv('test-public.csv')
ids = df_test_public['Id'].values
df_test_public.columns = testCols
df_test_public = df_test_public.drop('Id', axis = 1)
df_test_public

,source_node,destination_node
0,0,2917
1,0,2956
2,1,4038
3,2,1848
4,3,513
...,...,...
1995,3865,3924
1996,3917,4025
1997,3922,3947
1998,3955,3987


In [123]:
x_testing_n2v = [(n2w_model[str(i)]+n2w_model[str(j)]) for i,j in zip(df_test_public['source_node'], df_test_public['destination_node'])]

<ipython-input-123-2271668228b8>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  x_testing_n2v = [(n2w_model[str(i)]+n2w_model[str(j)]) for i,j in zip(df_test_public['source_node'], df_test_public['destination_node'])]


In [125]:
x_testing_n2v_1 = np.array(x_testing_n2v)

In [127]:
test_edges = df_test_public.to_numpy()

In [128]:
features_test = generate_features(test_edges, test=True)

features: 2000


In [129]:
def write_test_to_csv(features):
    with open("test.csv","w",newline="") as csvfile:
        writer=csv.writer(csvfile)
        writer.writerow(["JC","AA","Label"])
        writer.writerows(features)

In [130]:
write_test_to_csv(features_test)

In [131]:
dataset_test_sim = pd.read_csv('test.csv')
FEATURE_SIZE=2

X_test_sim = dataset_test_sim.iloc[:,:FEATURE_SIZE].values
y_test_sim = dataset_test_sim.iloc[:,FEATURE_SIZE].values

In [132]:
x_testing_sim = sc.transform(X_test_sim)

In [134]:
all_feats_testing = np.concatenate((x_testing_sim,x_testing_n2v_1),axis=1)

In [136]:
y = model.predict(all_feats_testing)

In [138]:
output = pd.DataFrame({'Id': ids, 'Predicted': y})
output.to_csv("LGBM_results_no_omissible_links.csv", index=False)